# 1. Introduction

Percolation is a simple model in statistical physics that has been well studied. It has a wide range of uses including modelling resistor networks, ecological disturbances (like forest fires), epidemics, and was first used to model fluid flow through porous materials.
<br><br> Percolation theory describes the behaviour of _clusters_ in a large lattice of dimension $d$. The two most studied percolation models are site percolation and bond percolation. In site percolation, the lattice vertices are the relevant quantitities. Each lattice vertex/site is either "occupied", with probability $p$, or "unoccupied", with probability 1-$p$. A cluster is a group of occupied lattice sites that are connected by a chain of nearest neigbour links (i.e for every site in a cluster, one of its nearest neigbours is also occupied) [1]. 
<br><br> Bond percolation is a similar model where the lattice bonds are the relevant quantities. Each bond is either "occupied", with probability $p$, or "unoccupied", with probability 1-$p$. In both models, the probability of a site/bond being occupied is independent of the status of its neighbours. The system is said to _percolate_ when the largest cluster spans the lattice (i.e. it extends from one lattice boundary to the opposite boundary).
<br><br>The critical proabibility value, $p_{c}$, is the probability at which the system can first percolate. For bond percolation in two dimensions, $p_{c} = \frac{1}{2}$. No analytic expression exists for site percolation in two dimensions, or for either model in 3 dimensions or above [2]. 
<br><br>Percolation is a random process. Different percolation lattices will contain clusters of different shapes and sizes - we wish to discuss their average properties. Hence, We often want to find the average value of some observable $Q(p)$ (e.g. average cluster size) over a range of values of $p$. In this report we will explore the most popular algorithm for doing this, the Newman-Ziff algorithm.


# 2. Percolation Algorithms

### 2.1 Conventional Algoritm

Let us first consider the conventional direct percolation algorithm. Consider some observable $Q(p)$ that we wish to calculate over a range of values of $p$. We must perform simulations at many closely spaced values of $p$, and interpolate between measurements to obtain a continous curve of $Q(p)$. This, however, introduces error into the results and it is better instead to measure $Q$ for a fixed number $n$ of occupied sites in the range of interest. The probability of there being exactly $n$ occupied sites out of a possible $N$ sites is given by the Binomial distribution:
<br>$$B(N,n,p) = \binom{N}{n}p^{n}(1-p)^{N-n}$$ 
If we measure the observable for all values of $n$ (giving a set of measurements {$Q_{n}$}) we can find $Q(p)$ for all $p$:
<br>$$Q(p)=\sum\limits_{n=0}^{N}B(N,n,p)Q_{n}$$
The same expression applies for bond percolation but replacing $N$ with $M$, where $M$ is the total number of bonds.
<br><br>For site percolation, starting with an empty lattice of $N$ sites and $M$ bonds , it takes time $O(N)$ to fill the lattice and time $O(M)$ to find all the clusters for each value of $n$. Hence, the calculation takes time $O(M+N)$ for each value of $n$ and so $O(N^{2}+MN)$ overall. For a regular lattice $M=\frac{1}{2}zN$, where $z$ is the co-ordination number, and so $O(N^{2}+MN)$ is equivalent to $O(N^{2})$ . The algorithm complexity for bond percolation is the same as for site percolation on a regular lattice. [2]

### 2.2 Newmann-Ziff Algorithm
The Newman-Ziff Algorithm takes time $O(N)$ which is a large improvement on the $O(N^{2})$ complexity of the conventional algorithm. 
<br><br>In the conventional percolation algorithm, a new state of the lattice with $n$ occupied sites/bonds is created for each value of $n$. However, since to measure $Q(p)$ we generate states for all values of $n$ from zero to $N$ then we can save time using the fact that a correct state with $n+1$ occupied sites/bonds can be derived from a correct state with $n$ occupied sites/bonds by adding one extra randomly chosen site/bond. The whole set of percolation states is derived from an empty lattice by adding sites/bonds one by one. This is the idea at the core of the algorithm. [3]
<br><br>Consider the case of bond percolation. We initially have an empty lattice with all $M$ bonds unoccupied so that each site is a cluster of size 1. To begin, we must randomly choose the order which the bonds are occupied. A simple way to do this is as follows:  
1. Create a list of the bonds with positions labelled from 1 to $M$
2. set $i=1$ and choose a number $j$ randomly in the range $1\le j\le M$ 
3. Exchange the bonds at positions $i$ and $j$
4. Repeat this for all $i$ in the range $1\le i\le M$

<br>This generates all possible permutations of bond orders with equal probability and has complexity $O(M)$.

#### __References__

[1] D. Stauffer and A. Aharony, _Introduction to Percolation Theory_, 2nd ed. Taylor and Francis, London, 1992
<br>[2] Newman, MEJ, and Robert M Ziff. 2000. “Efficient Monte Carlo Algorithm and High-Precision Results for Percolation.” _Physical Review Letters_ 85 (19): 4104.
<br>[3] Sorge, A. 2015. _The Newman–Ziff Algorithm pypercolate documentation_. Available at: https://pypercolate.readthedocs.io/en/stable/newman-ziff.html 